In [1]:
import math
import sys,os
import numpy as np
import pandas as pd

sys.path.append('/Users/Ziqi/Downloads/mgwr/')
from gwr.gwr import GWR,GWRResults
from gwr.sel_bw import Sel_BW
from spglm.family import Gaussian, Poisson, Binomial

In [2]:
zillow = pd.read_csv("Zillow-test-dataset/zillow_5k.csv",sep=',')
zillow.head()

,utmX,utmY,value,nbaths,nbeds,area,age
0,379979.957285,3.752562e+06,87.397,2.0,3.0,1311.0,61.0
1,411210.623578,3.762339e+06,121.211,2.0,4.0,1200.0,62.0
2,400287.486356,3.743925e+06,76.145,1.0,2.0,724.0,70.0
3,391029.680899,3.776406e+06,136.320,2.0,3.0,1890.0,54.0
4,371824.111160,3.750099e+06,100.499,4.0,5.0,2166.0,71.0


In [3]:
#Converting things into matrices
y = zillow.value.as_matrix().reshape(-1,1)
X = zillow.iloc[:,3:].as_matrix()
k = zillow.shape[1]
u = zillow.utmX
v = zillow.utmY
n = zillow.shape[0]
coords = list(zip(u,v))

In [4]:
%%time
bw = 500
pysal_Result=GWR(coords,y,X,bw).fit()

CPU times: user 26.4 s, sys: 1.72 s, total: 28.1 s
Wall time: 16.7 s


In [5]:
#Running FastGWR 1 Processor
!mpiexec -np 1 python fastgwr-mpi.py -data Zillow-test-dataset/zillow_5k.csv -out results.csv -a -c -bw 500

Starting FastGWR with 1 Processors
Data Input Path: Zillow-test-dataset/zillow_5k.csv
Output Result Path: results.csv
Constant: True
Spatial Kernel: Adaptive Bisquare
Fitting GWR Using Bandwidth: 500
Diagnostic Information:
AICc: 62399.20315246709
ENP: 132.5582097758877
R2: 0.6986624931885277
Total Time Elapsed: 3.09 seconds


In [6]:
#Running FastGWR 2 Processor
!mpiexec -np 2 python fastgwr-mpi.py -data Zillow-test-dataset/zillow_5k.csv -out results.csv -a -c -bw 500

Starting FastGWR with 2 Processors
Data Input Path: Zillow-test-dataset/zillow_5k.csv
Output Result Path: results.csv
Constant: True
Spatial Kernel: Adaptive Bisquare
Fitting GWR Using Bandwidth: 500
Diagnostic Information:
AICc: 62399.20315246709
ENP: 132.5582097758877
R2: 0.6986624931885277
Total Time Elapsed: 1.81 seconds


In [7]:
print("pysal AICc:", pysal_Result.aicc)
print("pysal R2:",pysal_Result.R2)

pysal AICc: 62399.20315246711
pysal R2: 0.6986624931885264


In [8]:
fastGWR_result = pd.read_csv("results.csv")
fastGWR_result.head()

,index,residual,influ,b_intercept,b_nbaths,b_nbeds,b_area,b_age,se_intercept,se_nbaths,se_nbeds,se_area,se_age
0,0.0,3.512057,0.010853,2.571375,2.978600,5.131739,0.067003,-0.457055,56.283078,18.213311,12.759484,0.029944,0.510628
1,1.0,72.505994,0.018748,25.630789,11.642206,-25.713499,0.145365,-1.157966,55.370680,16.996824,11.243051,0.017842,0.582930
2,2.0,37.792773,0.017085,70.665338,5.548814,-6.551175,0.093067,-1.316289,55.458494,16.724264,11.768216,0.021307,0.607088
3,3.0,-60.409974,0.013437,60.811811,3.897050,-24.768389,0.132632,-0.893417,38.141630,14.479511,10.784798,0.018262,0.347060
4,4.0,-124.192863,0.072594,-33.406094,14.888397,-15.429987,0.171549,-1.350430,59.658697,14.988391,11.275948,0.019607,0.642205


In [9]:
#Validate residual
np.allclose(fastGWR_result.residual,pysal_Result.resid_response.reshape(-1),atol=1e-4)

True

In [10]:
#Validate Params
np.allclose(np.array(fastGWR_result.iloc[:,3:8]),pysal_Result.params,atol=1e-3)

True

In [11]:
#Validate SEs
np.allclose(np.array(fastGWR_result.iloc[:,8:13]),pysal_Result.bse,rtol=1e-2)

True